In [1]:
# Dependencies
import pandas as pd

In [2]:
# Constants


In [3]:
# Aaron's Code

In [4]:
## Import Billboard Hits
filepath = "../data/external/billboard_hits/"
def import_billboard_hit(filename):
    columns = ["Date"]
    columns.extend(range(1,101))
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1", names=columns)

def import_billboard_feature(filename):
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1")

billboard_hits = []
billboard_features = []
for year in range(1960, 2020, 10):
    print(f"Importing {year}'s top 100.")
    billboard_hit = import_billboard_hit(f"billboard{year}")
    print(f"Importing {year}'s features.")
    billboard_feature = import_billboard_feature(f"billboard{year}FeaturesDatabase")
    
    billboard_hits.append(billboard_hit)
    billboard_features.append(billboard_feature)

Importing 1960's top 100.
Importing 1960's features.
Importing 1970's top 100.
Importing 1970's features.
Importing 1980's top 100.
Importing 1980's features.
Importing 1990's top 100.
Importing 1990's features.
Importing 2000's top 100.
Importing 2000's features.
Importing 2010's top 100.
Importing 2010's features.


In [5]:
billboard_hits[0].head()

,Date,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1960-01-04,"['El Paso', 'Marty Robbins']","['Why', 'Frankie Avalon']","['The Big Hurt', 'Miss Toni Fisher']","['Running Bear', 'Johnny Preston']","['Way Down Yonder In New Orleans', 'Freddie Ca...","['Heartaches By The Number', 'Guy Mitchell']","[""It's Time To Cry"", 'Paul Anka']","['Among My Souvenirs', 'Connie Francis']","['Pretty Blue Eyes', 'Steve Lawrence']",...,"[""I Don't Know What It Is"", 'The Bluenotes']","['Darling Lorraine', 'The Knockouts']","['In The Still Of The Nite', 'The Five Satins']","['Do-Re-Mi', 'Mitch Miller']","['Little Coco Palm', 'Jerry Wallace']","['Deck Of Cards', 'Wink Martindale']","['One Mint Julep', 'Chet Atkins']","['Happy Anniversary', 'Jane Morgan']","['Smokie-Part 2', 'Bill Doggett']","['One More Chance', 'Rod Bernard']"
1,1960-02-01,"['Running Bear', 'Johnny Preston']","['Teen Angel', 'Mark Dinning']","['El Paso', 'Marty Robbins']","['Where Or When', 'Dion & The Belmonts']","['Go, Jimmy, Go', 'Jimmy Clanton']","['Why', 'Frankie Avalon']","['Handy Man', 'Jimmy Jones']","['The Big Hurt', 'Miss Toni Fisher']","['Way Down Yonder In New Orleans', 'Freddie Ca...",...,"['Believe Me', 'Royal Teens']","['(There Was A) Tall Oak Tree', 'Dorsey Burnet...","['Since I Made You Cry', 'The Rivieras']","[""I'll Take Care Of You"", 'Bobby Bland']","[""Money (That's what I want)"", 'Barrett Strong']","['Peace Of Mind', 'Teresa Brewer']","['Fannie Mae', 'Buster Brown']","['(Baby) Hully Gully', 'The Olympics']","['Mediterranean Moon', 'The Rays']","['The Whiffenpoof Song', 'Bob Crewe']"
2,1960-02-29,"['The Theme From ""A Summer Place""', 'Percy Fai...","['Handy Man', 'Jimmy Jones']","[""He'll Have To Go"", 'Jim Reeves']","['Teen Angel', 'Mark Dinning']","[""What In The World's Come Over You"", 'Jack Sc...","['Beyond The Sea', 'Bobby Darin']","['Running Bear', 'Johnny Preston']","['Let It Be Me', 'The Everly Brothers']","[""Baby (You've Got What It Takes)"", 'Dinah Was...",...,"['Secret Of Love', 'Elton Anderson With Sid La...","['(Do The) Mashed Potatoes (Part 1)', 'Nat Ken...","['Let The Little Girl Dance', 'Billy Bland']","['Up Town', 'Roy Orbison']","[""You Don't Know Me"", 'Lenny Welch']","['Mountain Of Love', 'Harold Dorman']","['Road Runner', 'Bo Diddley']","['Among My Souvenirs', 'Connie Francis']","['The Old Payola Roll Blues (Side I)', 'Stan F...","['Just Give Me A Ring', 'Clyde McPhatter']"
3,1960-03-28,"['The Theme From ""A Summer Place""', 'Percy Fai...","['Wild One', 'Bobby Rydell']","[""He'll Have To Go"", 'Jim Reeves']","['Puppy Love', 'Paul Anka']","[""Sweet Nothin's"", 'Brenda Lee']","[""Baby (You've Got What It Takes)"", 'Dinah Was...","['Handy Man', 'Jimmy Jones']","['Harbor Lights', 'The Platters']","['Forever', 'The Little Dippers']",...,"['Road Runner', 'Bo Diddley']","['(Baby) Hully Gully', 'The Olympics']","['What Am I Living For', 'Conway Twitty']","['You Got What It Takes', 'Marv Johnson']","['Lonely Weekends', 'Charlie Rich']","[""Why I'm Walkin'"", 'Stonewall Jackson']","['Bad Boy', 'Marty Wilde']","['I Need You Now', '100 Strings and Jono (Choi...","['Stairway To Heaven', 'Neil Sedaka']","['Eternally', 'Sarah Vaughan']"
4,1960-04-25,"['Stuck On You', 'Elvis Presley With The Jorda...","['Greenfields', 'The Brothers Four']","['Sink The Bismark', 'Johnny Horton']","['The Theme From ""A Summer Place""', 'Percy Fai...","[""He'll Have To Go"", 'Jim Reeves']","['Sixteen Reasons', 'Connie Stevens']","['Puppy Love', 'Paul Anka']","['The Old Lamplighter', 'The Browns Featuring ...","['White Silver Sands', ""Bill Black's Combo""]",...,"['Someone Loves You, Joe', 'The Singing Belles']","[""Easy Lovin'"", 'Wade Flemons']","[""No If's - No And's"", 'Lloyd Price and His Or...","['Last Chance', 'Collay & the Satellites']","['Another Sleepless Night', 'Jimmy Clanton']","[""(Doin' The) Lovers Leap"", 'Webb Pierce']","['Two Thousand, Two Hundred, Twenty-Three Mile...","['Young Emotions', 'Ricky Nelson']","['Beautiful Obsession', 'Sir Chaunc

In [6]:
billboard_features[0].head()

,Track,Artist,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorusHit,sections
0,Sinner Man,Trini Lopez,spotify:track:7q0CKuWtZgTwOaNWDGtGVf,0.657,0.656,0.0,-9.806,1.0,0.0299,0.228,0.000000,0.1620,0.961,134.977,170920.0,4.0,NaN,NaN
1,Wichita Lineman,Glen Campbell,spotify:track:6V9VCm1zOY2lGR80RehJ9i,0.192,0.392,5.0,-11.667,1.0,0.0328,0.326,0.000199,0.2480,0.391,172.312,186200.0,4.0,34.89142,9.0
2,Nothing New (Same Old Thing),Fats Domino,spotify:track:3WgjedKdtPCwsA2qJ3IK0o,0.564,0.383,3.0,-9.386,1.0,0.0253,0.833,0.000008,0.0506,0.660,100.521,129227.0,3.0,19.06846,9.0
3,He Touched Me,Barbra Streisand,spotify:track:41nHzM661MpSZ2pVIXdZhB,0.243,0.347,8.0,-9.947,1.0,0.0327,0.916,0.000000,0.1780,0.201,102.429,189573.0,4.0,59.32625,8.0
4,Gimme Some Lovin',The Spencer Davis Group,spotify:track:3qCHUm73I3o5F1mDEfibzo,0.598,0.564,0.0,-6.454,1.0,0.0319,0.216,0.000204,0.2770,0.430,147.620,178840.0,4.0,28.14543,8.0


In [7]:
# Combine Hits Dataframes
all_billboard_hits = pd.concat(billboard_hits)
all_billboard_hits = all_billboard_hits.reset_index()
all_billboard_hits["Date"].nunique()

789

In [8]:
# Combine Features Dataframes
all_billboard_features = pd.concat(billboard_features)
all_billboard_features["Track"].nunique()

17371

In [9]:
# Split the date
split_dates = all_billboard_hits["Date"].str.split(pat = "-", expand=True)
split_dates.columns = ["Year", "Month", "Day"]
all_billboard_hits["Year"] = split_dates["Year"]
all_billboard_hits["Month"] = split_dates["Month"]
all_billboard_hits["Day"] = split_dates["Day"]
all_billboard_hits.head()

,index,Date,1,2,3,4,5,6,7,8,...,94,95,96,97,98,99,100,Year,Month,Day
0,0,1960-01-04,"['El Paso', 'Marty Robbins']","['Why', 'Frankie Avalon']","['The Big Hurt', 'Miss Toni Fisher']","['Running Bear', 'Johnny Preston']","['Way Down Yonder In New Orleans', 'Freddie Ca...","['Heartaches By The Number', 'Guy Mitchell']","[""It's Time To Cry"", 'Paul Anka']","['Among My Souvenirs', 'Connie Francis']",...,"['Do-Re-Mi', 'Mitch Miller']","['Little Coco Palm', 'Jerry Wallace']","['Deck Of Cards', 'Wink Martindale']","['One Mint Julep', 'Chet Atkins']","['Happy Anniversary', 'Jane Morgan']","['Smokie-Part 2', 'Bill Doggett']","['One More Chance', 'Rod Bernard']",1960,01,04
1,1,1960-02-01,"['Running Bear', 'Johnny Preston']","['Teen Angel', 'Mark Dinning']","['El Paso', 'Marty Robbins']","['Where Or When', 'Dion & The Belmonts']","['Go, Jimmy, Go', 'Jimmy Clanton']","['Why', 'Frankie Avalon']","['Handy Man', 'Jimmy Jones']","['The Big Hurt', 'Miss Toni Fisher']",...,"[""I'll Take Care Of You"", 'Bobby Bland']","[""Money (That's what I want)"", 'Barrett Strong']","['Peace Of Mind', 'Teresa Brewer']","['Fannie Mae', 'Buster Brown']","['(Baby) Hully Gully', 'The Olympics']","['Mediterranean Moon', 'The Rays']","['The Whiffenpoof Song', 'Bob Crewe']",1960,02,01
2,2,1960-02-29,"['The Theme From ""A Summer Place""', 'Percy Fai...","['Handy Man', 'Jimmy Jones']","[""He'll Have To Go"", 'Jim Reeves']","['Teen Angel', 'Mark Dinning']","[""What In The World's Come Over You"", 'Jack Sc...","['Beyond The Sea', 'Bobby Darin']","['Running Bear', 'Johnny Preston']","['Let It Be Me', 'The Everly Brothers']",...,"['Up Town', 'Roy Orbison']","[""You Don't Know Me"", 'Lenny Welch']","['Mountain Of Love', 'Harold Dorman']","['Road Runner', 'Bo Diddley']","['Among My Souvenirs', 'Connie Francis']","['The Old Payola Roll Blues (Side I)', 'Stan F...","['Just Give Me A Ring', 'Clyde McPhatter']",1960,02,29
3,3,1960-03-28,"['The Theme From ""A Summer Place""', 'Percy Fai...","['Wild One', 'Bobby Rydell']","[""He'll Have To Go"", 'Jim Reeves']","['Puppy Love', 'Paul Anka']","[""Sweet Nothin's"", 'Brenda Lee']","[""Baby (You've Got What It Takes)"", 'Dinah Was...","['Handy Man', 'Jimmy Jones']","['Harbor Lights', 'The Platters']",...,"['You Got What It Takes', 'Marv Johnson']","['Lonely Weekends', 'Charlie Rich']","[""Why I'm Walkin'"", 'Stonewall Jackson']","['Bad Boy', 'Marty Wilde']","['I Need You Now', '100 Strings and Jono (Choi...","['Stairway To Heaven', 'Neil Sedaka']","['Eternally', 'Sarah Vaughan']",1960,03,28
4,4,1960-04-25,"['Stuck On You', 'Elvis Presley With The Jorda...","['Greenfields', 'The Brothers Four']","['Sink The Bismark', 'Johnny Horton']","['The Theme From ""A Summer Place""', 'Percy Fai...","[""He'll Have To Go"", 'Jim Reeves']","['Sixteen Reasons', 'Connie Stevens']","['Puppy Love', 'Paul Anka']","['The Old Lamplighter', 'The Browns Featuring ...",...,"['Last Chance', 'Collay & the Satellites']","['Another Sleepless Night', 'Jimmy Clanton']","[""(Doin' The) Lovers Leap"", 'Webb Pierce']","['Two Thousand, Two Hundred, Twenty-Three Mile...","['Young Emotions', 'Ricky Nelson']","['Beautiful Obsession', 'Sir Chauncey and his ...","[""Dutchman's Gold"", 'Walter Brennan With Billy...",1960,04,25


In [10]:
# Reform the hits table to split up song/artist to be able to merge
def split_song_artist(song_artist):
    song_artist = song_artist.replace('"', "'")
    split = song_artist.split("', '")
    song = split[0].replace("['", "")
    artist = split[1].replace("']", "")
    return (song, artist)

In [11]:
song_datas = []
for index, row in all_billboard_hits.iterrows():
    for hit_num in range(1, 101):
        song_artist = row[hit_num]
        try:
            (song, artist) = split_song_artist(song_artist)
        except:
            print(f"Couldn't parse {song_artist}")
            song = ""
            artist = ""
        song_data = {
            "Track" : song,
            "Artist" : artist,
            "Placement" : hit_num,
            "Year" : row["Year"],
            "Month" : row["Month"],
            "Day" : row["Day"],
        }
        song_datas.append(song_data)

Couldn't parse nan
Couldn't parse nan
Couldn't parse nan
Couldn't parse nan


In [12]:
all_hits = pd.DataFrame(song_datas)
all_hits.head()

,Track,Artist,Placement,Year,Month,Day
0,El Paso,Marty Robbins,1,1960,01,04
1,Why,Frankie Avalon,2,1960,01,04
2,The Big Hurt,Miss Toni Fisher,3,1960,01,04
3,Running Bear,Johnny Preston,4,1960,01,04
4,Way Down Yonder In New Orleans,Freddie Cannon,5,1960,01,04


In [26]:
final_music = pd.merge(all_hits, all_billboard_features, on=["Track", "Artist"])
columns_to_drop = ["URI", "mode", "duration_ms", "instrumentalness", 
                   "time_signature", "chorusHit", "sections"]
final_music = final_music.drop(columns = columns_to_drop)
final_music = final_music.dropna()
final_music.head()

,Track,Artist,Placement,Year,Month,Day,danceability,energy,key,loudness,speechiness,acousticness,liveness,valence,tempo
0,El Paso,Marty Robbins,1,1960,01,04,0.654,0.452,2.0,-9.709,0.0300,0.835,0.16,0.691,106.662
1,El Paso,Marty Robbins,3,1960,02,01,0.654,0.452,2.0,-9.709,0.0300,0.835,0.16,0.691,106.662
2,El Paso,Marty Robbins,29,1960,02,29,0.654,0.452,2.0,-9.709,0.0300,0.835,0.16,0.691,106.662
3,El Paso,Marty Robbins,86,1960,03,28,0.654,0.452,2.0,-9.709,0.0300,0.835,0.16,0.691,106.662
4,Why,Frankie Avalon,2,1960,01,04,0.510,0.349,5.0,-8.677,0.0264,0.762,0.13,0.586,94.267


In [28]:
final_music.to_csv("../data/billboard_hits.csv")

In [15]:
# End of Aaron's Code

In [16]:
# Albin's Code

In [17]:
# End of Albin's Code

In [18]:
# Kelsey's Code

In [19]:
# End of Kelsey's Code

In [20]:
# Will's Code

In [21]:
# End of Will's Code